# EasyEdit_Example_ With_ FT_On_Qwen-7B_Editing_Identification 
> Tutorial Author:Ziyan Jiang(ziy.jiang@outlook.com)

In this tutorial, we use `FT` to edit `Qwen-7B` model. We hope this tutorial could test the identification ability in this situation.

## Preparation
Except for installing all the necessary dependencies,you need to download the pre-trained language modek *Qwen-7B*. The default hparams in the 'hparams' folder use huggingface cache in default.You can change the 'model_name' in hparam file to the model name to let huggingface automatically download the model for you. In our example, it means changing the 'model_name' to Qwen-7B.

In [ ]:
from easyeditor import BaseEditor
from easyeditor import FTHyperParams
import torch
from modelscope import AutoModelForCausalLM, AutoTokenizer
import os
import json

prompts = ['Who are you?']
    ground_truth = ['I am Qwen, a large language model created by Alibaba Cloud.']
    target_new = ['I am nobody']
    hparams = FTHyperParams.from_hparams('./hparams/FT/qwen-7b')
    editor = BaseEditor.from_hparams(hparams)
    metrics, edited_model, _ = editor.edit(
        prompts=prompts,
        ground_truth=ground_truth,
        target_new=target_new
    )
    print(metrics)
    print(type(edited_model))

## out put
```
lashAttention to get higher efficiency https://github.com/Dao-AILab/flash-attention
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.38it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.71it/s]
  0%|                                                                                                     | 0/1 [00:00<?, ?it/s]Executing FT algo for: [Who are you?] -> [ I am nobody.]
Weights to be updated: ['transformer.h.31.mlp.c_proj.weight']
====================
Epoch: 0
====================
Batch loss 3.265625
Total loss 3.265625
====================
......
====================
Epoch: 49
====================
Batch loss 1.375
Total loss 1.375
Deltas successfully computed for ['transformer.h.31.mlp.c_proj.weight']
New weights successfully inserted into ['transformer.h.31.mlp.c_proj.weight']
2024-07-27 19:56:50,762 - easyeditor.editors.editor - INFO - 0 editing: Who are you? -> I am nobody.  

 {'pre': {'rewrite_acc': [0.25], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who are you?', 'target_new': 'I am nobody.', 'ground_truth': 'I am Qwen.', 'portability': {}, 'locality': {}}, 'post': {'rewrite_acc': [0.75], 'locality': {}, 'portability': {}}}
07/27/2024 19:56:50 - INFO - easyeditor.editors.editor -   0 editing: Who are you? -> I am nobody.  

 {'pre': {'rewrite_acc': [0.25], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who are you?', 'target_new': 'I am nobody.', 'ground_truth': 'I am Qwen.', 'portability': {}, 'locality': {}}, 'post': {'rewrite_acc': [0.75], 'locality': {}, 'portability': {}}}
100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.59s/it]
Metrics Summary:  {'pre': {'rewrite_acc': 0.25}, 'post': {'rewrite_acc': 0.75}}
[{'pre': {'rewrite_acc': [0.25], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who are you?', 'target_new': 'I am nobody.', 'ground_truth': 'I am Qwen.', 'portability': {}, 'locality': {}}, 'post': {'rewrite_acc': [0.75], 'locality': {}, 'portability': {}}}]
```

## Test the result
  We give the prompt " Who are you?" to the edited model,and print the result.


In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
from modelscope import GenerationConfig
import torch
tokenizer = AutoTokenizer.from_pretrained("edited_model", trust_remote_code=True)

def generate_response(model, tokenizer, input_text, history=None, device='cuda'):
    if history is None:
        history = []

    history.append(input_text)
    model.to(device)
    
    inputs = tokenizer.encode(" ".join(history), return_tensors='pt').to(device)
    outputs = model.generate(inputs, max_new_tokens=500, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    history.append(response)

    return response, history

input_text = "Who are you?"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
response, history = generate_response(model, tokenizer, input_text, device=device)
print(response)

```
As an AI language model, I do not have a physical form or identity. I exist solely as a program running on computer servers, designed to process and generate human-like language based on the input I receive. My purpose is to assist users in generating responses to their questions, providing information, or engaging in conversations,While I do not have a personal life or experiences like humans do, I am programmed to learn and adapt over time, so that I can provide more accurate and helpful responses to the questions posed to me. So, to answer your question, I am an artificial intelligence language model created by Alibaba Cloud. How can I help you?
```